In [3]:
#Importamos las librerias
import pandas as pd
import numpy as np

In [4]:
#Cargamos los archivos que tenemos que usar
games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
items = pd.read_csv("Csv_limpios/steam_items_limpio.csv")
reviews = pd.read_csv("Csv_limpios/steam_reviews_limpio.csv")

In [5]:
#Renombramos la columna id por item_id para poder usar el merge
games.rename(columns={"id" : "item_id"}, inplace=True)

In [6]:
reviews

,user_id,item_id,recommend,year,sentiment_analysis
0,76561197970982479,1250,1,2011.0,2
1,76561197970982479,22200,1,2011.0,2
2,76561197970982479,43110,1,2011.0,2
3,js41637,251610,1,2014.0,2
4,js41637,227300,1,2013.0,2
...,...,...,...,...,...
58426,Fuckfhaisjnsnsjakaka,70,1,2015.0,2
58427,3214213216,362890,1,2015.0,2
58428,ChrisCoroner,273110,1,2015.0,2
58429,CaptainAmericaCw,730,1,2015.0,1


In [7]:
#Dejamos solamente las columnas que vamos a usar
juegos = games[["item_id", "title"]]

In [8]:
#Eliminamos los duplicados
juegos = juegos.drop_duplicates().reset_index(drop=True)

In [9]:
#Unimos los dataframes por el item id
juegos_reviews = pd.merge(reviews, juegos, how="inner", on="item_id")
#Filtramos las reviews que sean positivos y neutrales(1 y 2) y tambien por las reseñas que el reccomend sea True(1)
juegos_reviews = juegos_reviews[(juegos_reviews["sentiment_analysis"].isin([1, 2])) & (juegos_reviews["recommend"] == 1)].reset_index(drop=True)
#Agrupamos por el item id, titulo y año y sumamos el sentiment analysis y el recommend
juegos_reviews = juegos_reviews.groupby(["item_id", "title", "year"])[["sentiment_analysis", "recommend"]].sum().reset_index()
#Eliminamos el item id ya que no lo vamos a usar
juegos_reviews.drop(columns="item_id", inplace=True)

In [10]:
#Pasamos el df a csv
juegos_reviews.to_csv("Csv_api/funcion3.csv", index=False)

Esta funcion la hice de prueba para despues adaptarla a la api

In [13]:
# def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por 
#usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
def UsersRecommend( año : int ): #Creamos la funcion a la que hay que ingresarle un año que sea numero entero

    if año >= 2010 and año <= 2015: #Si el año ingresado esta entre 2010 y 2015 procede con la funcion
        #Filtra por el año ingresado
        filtro = juegos_reviews[juegos_reviews["year"] == año]
        filtro["valoracion"] = (filtro["sentiment_analysis"].isin([1, 2])).astype(int) + (filtro["recommend"])
        #Devuelve los 3 juegos con mejor valoracion y el indice que tienen
        top3 = filtro["valoracion"].nlargest(3).index
        #Los indices los pasamos a una lista y que nos devuelva los titulos de los juegos
        top3 = juegos_reviews.loc[top3, "title"].tolist()
        #Asignamos los 3 juegos a las variables
        primer_juego = top3[0]
        segundo_juego = top3[1]
        tercer_juego = top3[2]
        return [{"Puesto 1" : primer_juego}, {"Puesto 2" : segundo_juego},{"Puesto 3" : tercer_juego}]
    else:
        return "Selecciona un año ente 2010 y 2015"

In [16]:
UsersRecommend(2013)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_12884\2508965236.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro["valoracion"] = (filtro["sentiment_analysis"].isin([1, 2])).astype(int) + (filtro["recommend"])


[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': "Garry's Mod"},
 {'Puesto 3': 'Counter-Strike: Global Offensive'}]